<a href="https://colab.research.google.com/github/sanAkel/HAFS_Air-Sea/blob/main/getStormTrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Retrives the track of a single hurricane from tropycal. Also saves track subsetted by category or intensitifcation rate.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Set up environment.

Track data comes from [tropycal](https://tropycal.github.io/tropycal/install.html).

In [ ]:
!pip install tropycal
!pip install cartopy

In [ ]:
from tropycal import tracks, rain
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

User Inputs

In [ ]:
# Basin and year
myBasin = 'north_atlantic'
year = 2024
name = "Milton"

intThreshold = 96 # Threshold for a major hurricane in kt
riThreshold = 30 # Intensification threshold in riTime for RI in kt
riTime = 24 # in hr

Retrieve storm

In [ ]:
# Initialize
# https://tropycal.github.io/tropycal/api/generated/tropycal.tracks.TrackDataset.html#tropycal.tracks.TrackDataset

basin = tracks.TrackDataset(basin=myBasin, source='hurdat', include_btk=True) # include most recent season
season = basin.get_season(year)
tc = season.get_storm((name, year))
print(tc)

Plot TC track

In [ ]:
oPath = f"/content/drive/MyDrive/tmp/{name}"

!mkdir -p {oPath}

tc.plot(map_prop={'figsize':(8,5),'linewidth':0.5, 'state_alpha':0.5})
plt.savefig(f'{oPath}/hurdat2_{name}.png')

Save whole storm track

In [ ]:
tc_DF = tc.to_dataframe()
tc_DF.to_csv(f'{oPath}/hurdat2_{name}.csv')

In [ ]:
tc_DF

Subset by category

In [ ]:
tcMajor = tc_DF[tc_DF.vmax >= intThreshold].reset_index()
tcMajor.to_csv(f"{oPath}/hurdat2_{name}_major.csv")

In [ ]:
tcMajor

Subset by Rapid Intensification

In [ ]:
#This only works for TCs with a single RI episode in its lifetime
riStart = None
riEnd = None

for row in tc_DF.iloc:
  futureTime = row.time + pd.Timedelta(hours=riTime)

  if futureTime in list(tc_DF.time):
    intensification = float(tc_DF[tc_DF.time == futureTime].vmax - row.vmax)
    if intensification >= riThreshold and riStart == None: riStart = row.time
    #if intensification < riThreshold and riStart != None and riEnd == None: riEnd = row.time

In [ ]:
riStart

In [ ]:
riEnd = max(tc_DF[tc_DF.vmax==max(tc_DF.vmax)].time) #This only works for a TC where RI ends once the TC reaches its peak intensity

In [ ]:
riEnd

In [ ]:
postRIStart = tc_DF[tc_DF.time >= riStart]
tcRI = postRIStart[postRIStart.time <= riEnd].reset_index()
tcRI.to_csv(f"{oPath}/hurdat2_{name}_RI.csv")

In [ ]:
tcRI

In [ ]:
# @title time vs mslp

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['time']
  ys = series['mslp']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = tcRI.sort_values('time', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('type')):
  _plot_series(series, series_name, i)
  fig.legend(title='type', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('time')
_ = plt.ylabel('mslp')